In [1]:
import plotly.graph_objs as go
from ipywidgets import widgets
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget
import numpy as np

N = 100
x = np.linspace(0, 1, N)
y_orig = np.random.randn(N)+5
y_new = y_orig

def resetValues(b):
    f.data[0].x = x
    f.data[0].y = y_orig
    scatter = f.data[0]
    colors = ['#a3a7e4'] * 100
    scatter.marker.color = colors
    scatter.marker.size = [10] * 100
    f.layout.hovermode = 'closest'

reset_bt = widgets.Button(description="Reset")

text_input = widgets.Text(
    description='y:',
    value='0',
)

reset_bt.on_click(resetValues)

container = widgets.HBox(children=[text_input, reset_bt])
display(container)

f = go.FigureWidget([go.Scatter(x=x, y=y_orig)])

scatter = f.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = 'closest'

# create our callback function
def update_point(trace, points, selector):
    print(y_orig)
    y_new[points.point_inds] = float(text_input.value)
    f.data[0].x = x
    f.data[0].y = y_new
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#f23535'
        s[i] = 20
        scatter.marker.color = c
        scatter.marker.size = s
        print(c)
    print(y_orig)

scatter.on_click(update_point)

f

<IPython.core.display.Javascript object>

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
               …

[5.26232887 4.29801528 5.42013384 5.29334539 6.9315211  5.07230938
 5.56580399 5.8393443  5.05761594 6.33168322 6.38482851 5.15256978
 4.08086683 4.53575908 5.59276778 3.19854478 5.37266376 5.37981041
 5.13890503 4.86902136 4.78802439 3.62512947 5.03583102 5.2781581
 4.07854681 5.50844373 5.77285305 5.66503051 4.56169576 5.03881337
 5.85165664 5.23767598 3.25619468 4.60201253 4.99812814 5.77589822
 5.08151769 4.63970259 5.35107549 5.69184603 3.80146679 6.69476604
 5.3571983  4.6992435  4.0263129  3.61385682 3.42311718 3.88724481
 3.89989005 6.649522   3.26536414 6.8226783  6.01993264 6.34033897
 5.59458754 5.3039465  5.58612147 3.75404594 7.28579844 6.21607233
 4.76123834 4.22186645 5.408654   5.46337218 5.01623305 4.98412548
 4.93335208 6.04434453 6.04325415 6.90111127 4.60747637 4.81421019
 4.28963898 5.76592826 4.73828304 5.51511301 6.18256041 5.1602828
 4.53052604 4.54563419 3.04338654 4.78533915 5.41948174 5.24878216
 5.9695964  6.09612752 5.09559072 5.00654021 5.97906525 5.058269